The following 2 cells access the PDF stored in the 'docs' directory, extract text from it, and convert it into JSON format indexed by page. 

In [5]:
import fitz  # PyMuPDF
from pathlib import Path

def extract_text_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    page_texts = []
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text = page.get_text()
        page_texts.append(text)
    return page_texts

pdf=Path("docs/QGenda Whitepaper.pdf")

raw_text = extract_text_from_pdf(pdf)
# print(raw_text)


In [6]:
# Pagination
import re
import json

# Regular expression pattern to match copyright text
pattern = r"Copyright © 2024 QGenda, LLC All rights reserved."

# Split data after each occurrence of the pattern
pages = []
current_page = []
for index, item in enumerate(raw_text):
    current_page.append(item)
    if re.search(pattern, item):
        pages.append({"index": len(pages), "text": "".join(current_page)})
        current_page = []

# Convert the pages list to JSON format
pages_json = json.dumps(pages, indent=4)

# Output the result
# print(pages_json)

Now, we clean the data the lazy way. Use a GPT from OpenAI to transcrobe the important text and remove symbols that are artifacts of encoding.

In [7]:
from langchain.prompts import PromptTemplate

template = """
Given the following extracted parts of a PDF document ("SOURCES") identify the primary text of the document (i.e. ignore text like 'Copyright © 2024' and 'n19%') and transcribe it.
Transcribe it strictly and do not add any other boiler plate text to you answer. It should just be text from the document. 
REMOVE encoding symbols for new lines (\n and \n\n), and make sure to look for this pattern even if they're touching another word. 
SOURCES:
{summaries}
=========
ANSWER:
"""
prompt = PromptTemplate(template=template, input_variables=["summaries"])

In [12]:
# Lazy way: Use LLM to clean up outputs.
from langchain_openai import OpenAI
from langchain.docstore.document import Document
from langchain.chains import LLMChain
from utilities import get_openai_key

api_key = get_openai_key()

llm = OpenAI(temperature=1.0)
cleaned_pages = []
for page in pages:
    raw_text = page['text']
    # Create the chain using the RunnableSequence
    chain_qa = prompt | llm
    llm_results = chain_qa.invoke({"summaries": raw_text}, return_only_outputs=True)
    cleaned_text = llm_results
    cleaned_pages.append({"index": page['index'], "text": cleaned_text})

# Convert the cleaned pages list to JSON format
cleaned_pages_json = json.dumps(cleaned_pages, indent=4)

# Output the result
print(cleaned_pages_json)

OpenAI key accessed.
[
    {
        "index": 0,
        "text": "Improving Capacity and Revenue through Effective Room Management Healthcare Leaders Experiencing Exam Room Utilization Challenges, Leading to Revenue Loss"
    },
    {
        "index": 1,
        "text": "As healthcare leaders reimagine patient access and care delivery in a post-pandemic world, there is an emerging story around the need to optimize clinic exam room utilization. Better utilization can improve operating efficiency, patient satisfaction, and revenue capture. The 2020 Porter Research study of 100 health system executive leaders identified the many challenges faced today with exam room scheduling, future expectations for optimizing exam rooms, and the impact that proper exam room scheduling can have on a health system\u2019s P&L. Increasing Scrutiny on Clinic Exam Rooms. With the limited funds available for health system capital expenditures, executives must optimize their existing physical space while simul

Create a .json file of the cleaned data for later use. 

In [13]:
from pathlib import Path

original_f = Path(pdf)
new_f = original_f.with_stem(original_f.stem + '_cleaned')
json_format = new_f.with_suffix('.json')

with open(json_format, 'w', encoding='utf-8') as file:
    file.write(cleaned_pages_json)